## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"
***

## Тестирование сервисов

__Постановка задачи__

Напишите FastAPI-микросервис для выдачи рекомендаций, который:
- принимает запрос с идентификатором пользователя и выдаёт рекомендации,
- учитывает историю пользователя,
- смешивает онлайн- и оффлайн-рекомендации.

В README опишите стратегию смешивания оффлайн- и онлайн-рекомендаций.

Оформите тестирование микросервиса в скрипте `test_service.py`
- для пользователя без персональных рекомендаций,
- для пользователя с персональными рекомендациями, но без онлайн-истории,
- для пользователя с персональными рекомендациями и онлайн-историей.

Выполните скрипт `test_service.py` для тестирования микросервиса, сохраните его вывод в файле `test_service.log`

### Инициализация

In [1]:
import numpy as np
import pandas as pd
import requests

Задаем url-адреса трех веб-сервисов

In [2]:
# Основной сервис для получения оффлайн- и онлайн-рекомендаций
recommendations_url = "http://127.0.0.1:8000"

# Вспомогательный сервис для получения рекомендаций по умолчанию на основе топ-треков
features_store_url = "http://127.0.0.1:8010"

# Вспомогательный сервис для хранения и получения последних онлайн-событий пользователя
events_store_url = "http://127.0.0.1:8020"

In [3]:
# Общий заголовок для всех http-запросов
headers = {"Content-type": "application/json", "Accept": "text/plain"}

Чтобы запустить тестируемые сервисы, нужно выполнить 4 команды (по одному на каждый сервис) в 4-х разных терминалах, находясь в папке проекта:
```
uvicorn recommendations_service:app
uvicorn events_service:app --port 8020
uvicorn features_service:app --port 8010
uvicorn test_service:app --port 8030
```

### Загрузка данных

Загрузким треки для отображения рекомендаций со всеми полями, а также сами рекомендации, чтобы выбирать из них пользователей и объекты для тестирования

In [4]:
items = pd.read_parquet("items.parquet")
recommendations = pd.read_parquet('recommendations.parquet')
similar = pd.read_parquet('similar.parquet')
top_popular = pd.read_parquet('top_popular.parquet')

### Получение рекомендаций по умолчанию

В этом сценарии мы получаем рекомендации на основе топ-треков, когда у пользователя нет истории и соответственно нет персональных оффлайн-рекомендаций.

Напишем вспомогательную функцию для отображения объектов по их идентификаторам

In [5]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "name", "genres", "albums", "artists"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)

Посмотрим топ-рекомендации из соответствующего файла

In [6]:
top_popular.head(10)

,rank,item_id,plays,users,name,genres,artists,albums
0,1,53404,40905,40905,Smells Like Teen Spirit,"[allrock, alternative, rock]",[Nirvana],"[Skiing Music, Nirvana, Nevermind, Smells Like..."
1,2,178529,35411,35411,Numb,"[numetal, metal]",[Linkin Park],"[Meteora, 00s Rock Anthems]"
2,3,33311009,34005,34005,Believer,"[allrock, rock]",[Imagine Dragons],"[Summertime Love, Family Friendly Summer Hits,..."
3,4,35505245,32259,32259,I Got Love,"[rusrap, rap]","[Miyagi & Эндшпиль, Рем Дигга]",[I Got Love]
4,5,37384,30390,30390,Zombie,"[allrock, rock]",[The Cranberries],"[***** ***, Gold, MNM Sing Your Song: Back To ..."
5,6,24692821,28508,28508,Way Down We Go,[indie],[KALEO],"[DFM Dance 8, Hits and Stars 2018, Chilled Aco..."
6,7,795836,28285,28285,Shape Of My Heart,"[allrock, pop, rock]",[Sting],"[Ten Summoner's Tales, 25 Years, Сентиментальн..."
7,8,6705392,26781,26781,Seven Nation Army,[alternative],[The White Stripes],"[Seven Nation Army, Radio 1 Sonar 1000, Pay Cl..."
8,9,32947997,26688,26688,Shape of You,[pop],[Ed Sheeran],"[Pop, Summer Vibes, ÷, Shape of You]"
9,10,45499814,26299,26299,Life,"[ruspop, pop]",[Zivert],"[Vinyl #1, Made in Russia, Vinyl #1. Deluxe Ve..."


Отправим запрос для получения рекомендаций по умолчанию и сравним его резульат с данными выше

In [7]:
params = {'k': 10}
resp = requests.post(recommendations_url + "/recommendations_default", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,53404,Smells Like Teen Spirit,"[allrock, alternative, rock]","[Skiing Music, Nirvana, Nevermind, Smells Like...",[Nirvana]
1,178529,Numb,"[numetal, metal]","[Meteora, 00s Rock Anthems]",[Linkin Park]
2,33311009,Believer,"[allrock, rock]","[Summertime Love, Family Friendly Summer Hits,...",[Imagine Dragons]
3,35505245,I Got Love,"[rusrap, rap]",[I Got Love],"[Miyagi & Эндшпиль, Рем Дигга]"
4,37384,Zombie,"[allrock, rock]","[***** ***, Gold, MNM Sing Your Song: Back To ...",[The Cranberries]
5,24692821,Way Down We Go,[indie],"[DFM Dance 8, Hits and Stars 2018, Chilled Aco...",[KALEO]
6,795836,Shape Of My Heart,"[allrock, pop, rock]","[Ten Summoner's Tales, 25 Years, Сентиментальн...",[Sting]
7,6705392,Seven Nation Army,[alternative],"[Seven Nation Army, Radio 1 Sonar 1000, Pay Cl...",[The White Stripes]
8,32947997,Shape of You,[pop],"[Pop, Summer Vibes, ÷, Shape of You]",[Ed Sheeran]
9,45499814,Life,"[ruspop, pop]","[Vinyl #1, Made in Russia, Vinyl #1. Deluxe Ve...",[Zivert]


Результаты запроса и данные в файле совпадают.

### Получение персональных рекомендаций без онлайн-истории

В этом сценарии рекомендации извлекаются по идентификтору пользователя из данных, сгенерированных на основании истории пользователя.

In [8]:
# Выберем случайного пользователя из оффлайн-рекомендаций
user_id = recommendations['user_id'].sample().iat[0]
print(f"user_id выбранного пользователя: {user_id}")

user_id выбранного пользователя: 617032


In [9]:
# Посмотрим, какие для него есть рекомендации
recommendations.query('user_id == @user_id').head(10)

,user_id,item_id,als_score,tracks_played_by_user,genre_0,genre_20,genre_2,genre_11,genre_6,genre_74,genre_21,genre_4,genre_14,genre_10,genre_others,score,rank
30810700,617032,32947997,1.012401,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.999968,1
30810701,617032,29544272,0.837257,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.994152,2
30810702,617032,18385776,0.456379,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.834461,3
30810703,617032,34976783,0.268101,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.374376,4
30810704,617032,26918469,0.196452,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.246755,5
30810705,617032,31219146,0.193369,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.235464,6
30810706,617032,31746480,0.191059,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.235464,7
30810707,617032,24663745,0.179783,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.214850,8
30810708,617032,30049955,0.177514,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.214850,9
30810709,617032,21497837,0.174815,63.0,0.37619,0.050794,0.0,0.092593,0.0,0.015873,0.034921,0.0,0.164021,0.007937,0.257672,0.214850,10


Выполним запрос и сравним его результат с данными выше

In [18]:
# Отправляем запрос для получения только оффлайн-рекомендаций
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,32947997,Shape of You,[pop],"[Pop, Summer Vibes, ÷, Shape of You]",[Ed Sheeran]
1,29544272,Human,"[indie, soundtrack]","[Some Chillout Music, Music Made for Chillout,...",[Rag'n'Bone Man]
2,18385776,Freaks,"[house, dance, electronics]","[Танцевальный рай: Hit Mix, Wild Nights 2018, ...","[Savage, Timmy Trumpet]"
3,34976783,Thunder,"[allrock, rock]","[Spring Breakers 2021, Clean Kids Party Mix, K...",[Imagine Dragons]
4,26918469,We Don't Talk Anymore,[pop],"[Nine Track Mind, Autumn Music 2016]","[Selena Gomez, Charlie Puth]"
5,31219146,My Way,[dance],"[Летний Хит FM, My Way, I Love Skiing, ADM: Ac...",[Calvin Harris]
6,31746480,Rockabye,"[pop, dance]","[DFM Dance 8, Pop Classics, Spring Music 2017,...","[Clean Bandit, Anne-Marie, Sean Paul]"
7,24663745,Ocean Drive,"[dance, electronics]","[Best Of The 10's: Dance, Pop Motivation, Work...",[Duke Dumont]
8,30049955,Lost on You,"[pop, dance]","[NOW! That's What I Call Music, Радио и интерн...",[LP]
9,21497837,Worth It,[pop],"[How To Be Single, Keep Calm & Party, Party Mi...","[Kid Ink, Fifth Harmony]"


По `item_id` видим, что результаты запроса и данные в файле совпадают.

### Получение персональных рекомендаций с онлайн-историей

В этом сценарии к оффлайн-рекомендациям из предыдущего раздела добавляются объекты, похожие на 3 последних прослушанных трека из онлайн-истории.

Возьмем 3 произвольных трека из i2i-рекомендаций и добавим их в онлайн-историю того же пользователя, что и в предыдущем тестовом сценарии

In [19]:
# Выбираем 3 объекта из i2i-рекомендаций
item_ids = similar['item_id_1'].unique()[:3]
print(item_ids)

# Добавляем их в онлайн-историю пользователя
for item_id in item_ids:
    params = {"user_id": user_id, "item_id": item_id}
    resp = requests.post(events_store_url + "/put", headers=headers, params=params)
    if resp.status_code == 200:
        resp = resp.json()
        print(resp['result'])    
    else:
        print(f"status code: {resp.status_code}") 

[ 99262 590303 597196]
ok
ok
ok


Проверим, что события добавились, причем в обратном порядке

In [20]:
params = {"user_id": user_id, "k": 3}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    print(resp['events'])    
else:
    print(f"status code: {resp.status_code}")

[597196, 590303, 99262]


Протестируем работу сервиса `feautures_service`. 

Возмьмем последний объект из списка выше и найдем похожие на него треки из файла `similar.parquet`

In [21]:
item_id_1 = item_ids[-1]
similar.query('item_id_1 == @item_id_1').head(10)

,score,item_id_1,item_id_2
23,0.953137,597196,21101461
24,0.945990,597196,21101463
25,0.923976,597196,597227
26,0.921177,597196,2222537
27,0.911217,597196,24143071
28,0.909582,597196,38788303
29,0.908665,597196,21101459
30,0.906829,597196,27323612
31,0.905836,597196,2836249
32,0.904213,597196,597201


Теперь получим тот же набор объектов с помощью сервиса `features_service`

In [22]:
params = {"item_id": item_id_1, "k": 10}
resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    print(resp['item_id_2'])    
else:
    print(f"status code: {resp.status_code}")    

[21101461, 21101463, 597227, 2222537, 24143071, 38788303, 21101459, 27323612, 2836249, 597201]


Видим, что данные совпадают.

Протестируем запрос для получения объектов, похожих на 3 последних трека из онлайн-истории выбранного пользователя

In [23]:
params = {'user_id': user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    recs = []
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,590413,В Багдаде всё спокойно,"[ruspop, pop]","[Car-Mania, «Физрук»]",[Кар-Мэн]
1,21101461,Ды-ды-дым,"[rusrap, rap]",[Лучшие песни],[Каста]
2,590668,Чио-Чио-сан,"[ruspop, pop]",[Вокруг света],[Кар-Мэн]
3,590164,Париж,"[ruspop, pop]",[Вокруг света],[Кар-Мэн]
4,21101463,Сочиняй мечты,"[rusrap, rap]",[Лучшие песни],[Каста]
5,590262,Бомбей буги,"[ruspop, pop]","[Car-Mania, «Физрук»]",[Кар-Мэн]
6,590692,Великий инквизитор,"[ruspop, pop]","[Русская массированная звуковая агрессия, «Физ...",[Кар-Мэн]
7,591001,Багама-Мама,"[ruspop, pop]",[Вокруг света],[Кар-Мэн]
8,99245,Ragga Steady,[pop],"[Midi, Maxi & Efti]","[Midi, Maxi & Efti]"
9,590326,Девушка с Карибских островов,"[ruspop, pop]",[Car-Mania],[Кар-Мэн]


Видим, что результат содержит в т.ч. объекты, похожие на последий трек из онлайн-истории пользователя.

Наконец, получим смешанные рекомендации. На нечетных местах будут оффлайн-рекомендации, на четных - онлайн.

In [24]:
params = {"user_id": user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    resp = resp.json()
    display_items(resp['recs'])
else:
    print(f"status code: {resp.status_code}")

,item_id,name,genres,albums,artists
0,32947997,Shape of You,[pop],"[Pop, Summer Vibes, ÷, Shape of You]",[Ed Sheeran]
1,590413,В Багдаде всё спокойно,"[ruspop, pop]","[Car-Mania, «Физрук»]",[Кар-Мэн]
2,29544272,Human,"[indie, soundtrack]","[Some Chillout Music, Music Made for Chillout,...",[Rag'n'Bone Man]
3,21101461,Ды-ды-дым,"[rusrap, rap]",[Лучшие песни],[Каста]
4,18385776,Freaks,"[house, dance, electronics]","[Танцевальный рай: Hit Mix, Wild Nights 2018, ...","[Savage, Timmy Trumpet]"
5,590668,Чио-Чио-сан,"[ruspop, pop]",[Вокруг света],[Кар-Мэн]
6,34976783,Thunder,"[allrock, rock]","[Spring Breakers 2021, Clean Kids Party Mix, K...",[Imagine Dragons]
7,590164,Париж,"[ruspop, pop]",[Вокруг света],[Кар-Мэн]
8,26918469,We Don't Talk Anymore,[pop],"[Nine Track Mind, Autumn Music 2016]","[Selena Gomez, Charlie Puth]"
9,21101463,Сочиняй мечты,"[rusrap, rap]",[Лучшие песни],[Каста]
